<a href="https://colab.research.google.com/github/amankhullar/cs20/blob/master/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> CS20 Assignment 1 </h1> </center>
## Problem 1: Op is all you need

In [0]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVL'] = '2'

import tensorflow as tf

sess = tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
sess = tf.InteractiveSession()

x = tf.random_uniform([])
y = tf.random_uniform([])

op = tf.cond(x > y, lambda : x + y, lambda : x - y)

print(sess.run(op))

1.1196562


In [0]:
x = tf.random_uniform(shape = [], minval = -1, maxval = 1, name = "x")
y = tf.random_uniform(shape = [], minval = -1, maxval = 1, name = "y")

# Using a tf.Tensor as a Python 'bool' is not allowed
# out = tf.zeros(shape = [1])

# if tf.less(x, y):
#     out = x + y
# elif tf.greater(x, y):
#     out = x - y
# else:
#     out = 0
    
def f1():
    return x + y

def f2():
    return x - y

def f3():
    return tf.zeros(shape = [1])

out = tf.case({tf.less(x, y) : f1, tf.greater(x, y) : f2}, default = f3, name = 'output', exclusive = True)
print(sess.run(out))

0.27322483


In [0]:
x = tf.constant([[0, -2, -1], [0, 1, 2]])
y = tf.zeros_like(x)

out = tf.equal(x, y, name = 'equal_output')

print(sess.run(out))

[[ True False False]
 [ True False False]]


In [0]:
sess = tf.InteractiveSession()

data = tf.constant([29.05088806,  27.61298943,  31.19073486,  29.35532951,
                     30.97266006,  26.67541885,  38.08450317,  20.74983215,
                     34.94445419,  34.45999146,  29.06485367,  36.01657104,
                     27.88236427,  20.56035233,  30.20379066,  29.51215172,
                     33.71149445,  28.59134293,  36.05556488,  28.66994858])

out = tf.gather(data, tf.where(tf.greater(data, 30)))

print(sess.run(out))

[[31.190735]
 [30.97266 ]
 [38.084503]
 [34.944454]
 [34.45999 ]
 [36.01657 ]
 [30.20379 ]
 [33.711494]
 [36.055565]]


In [0]:
# Numpy method of creating the diagonal matrix

# data = np.arange(1,7)
# out = np.diag(data)

# out_tf = tf.constant(out)

# print(sess.run(out_tf))

#Tensorflow method

data = tf.range(start = 1, limit = 7, name = 'range')
out = tf.linalg.diag(data, name = 'diagonal')

print(sess.run(out))

[[1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]
 [0 0 0 0 5 0]
 [0 0 0 0 0 6]]


In [0]:
data = tf.random_normal(shape = [10, 10], name = 'matrix')

out = tf.matrix_determinant(data, name = 'determinant')

print(sess.run(out))

-66.201225


In [0]:
x = tf.constant([5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9])

out, idx = tf.unique(x)

print(sess.run(out))

[ 5  2  3 10  6  4  1  0  9]


In [0]:
x = tf.random_normal(shape = [300], name = 'x')
y = tf.random_normal(shape = [300], name = 'y')

out = tf.cond(tf.reduce_mean(x - y) < 0, lambda : tf.losses.mean_squared_error(labels = x, predictions = y), lambda : tf.reduce_sum(tf.abs(x - y)))

print(sess.run(out))

2.150824


## Problem 2: Logistic Regression

### Utils

In [0]:
import os
import gzip
import shutil
import struct
import urllib

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

def huber_loss(labels, predictions, delta=14.0):
    residual = tf.abs(labels - predictions)
    def f1(): return 0.5 * tf.square(residual)
    def f2(): return delta * residual - 0.5 * tf.square(delta)
    return tf.cond(residual < delta, f1, f2)

def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

def read_birth_life_data(filename):
    """
    Read in birth_life_2010.txt and return:
    data in the form of NumPy array
    n_samples: number of samples
    """
    text = open(filename, 'r').readlines()[1:]
    data = [line[:-1].split('\t') for line in text]
    births = [float(line[1]) for line in data]
    lifes = [float(line[2]) for line in data]
    data = list(zip(births, lifes))
    n_samples = len(data)
    data = np.asarray(data, dtype=np.float32)
    return data, n_samples

def download_one_file(download_url, 
                    local_dest, 
                    expected_byte=None, 
                    unzip_and_remove=False):
    """ 
    Download the file from download_url into local_dest
    if the file doesn't already exists.
    If expected_byte is provided, check if 
    the downloaded file has the same number of bytes.
    If unzip_and_remove is True, unzip the file and remove the zip file
    """
    if os.path.exists(local_dest) or os.path.exists(local_dest[:-3]):
        print('%s already exists' %local_dest)
    else:
        print('Downloading %s' %download_url)
        local_file, _ = urllib.request.urlretrieve(download_url, local_dest)
        file_stat = os.stat(local_dest)
        if expected_byte:
            if file_stat.st_size == expected_byte:
                print('Successfully downloaded %s' %local_dest)
                if unzip_and_remove:
                    with gzip.open(local_dest, 'rb') as f_in, open(local_dest[:-3],'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    os.remove(local_dest)
            else:
                print('The downloaded file has unexpected number of bytes')

def download_mnist(path):
    """ 
    Download and unzip the dataset mnist if it's not already downloaded 
    Download from http://yann.lecun.com/exdb/mnist
    """
    safe_mkdir(path)
    url = 'http://yann.lecun.com/exdb/mnist'
    filenames = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
    expected_bytes = [9912422, 28881, 1648877, 4542]

    for filename, byte in zip(filenames, expected_bytes):
        download_url = os.path.join(url, filename)
        local_dest = os.path.join(path, filename)
        download_one_file(download_url, local_dest, byte, True)

def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k':
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte')
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8) #int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1
    
    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols) #uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels

def read_mnist(path, flatten=True, num_train=55000):
    """
    Read in the mnist dataset, given that the data is stored in path
    Return two tuples of numpy arrays
    ((train_imgs, train_labels), (test_imgs, test_labels))
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0])
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return (train_img, train_labels), (val_img, val_labels), test

def get_mnist_dataset(batch_size):
    # Step 1: Read in data
    mnist_folder = 'data/mnist'
    download_mnist(mnist_folder)
    train, val, test = read_mnist(mnist_folder, flatten=False)

    # Step 2: Create datasets and iterator
    train_data = tf.data.Dataset.from_tensor_slices(train)
    train_data = train_data.shuffle(10000) # if you want to shuffle your data
    train_data = train_data.batch(batch_size)

    test_data = tf.data.Dataset.from_tensor_slices(test)
    test_data = test_data.batch(batch_size)

    return train_data, test_data
    
def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    plt.imshow(image, cmap='gray')
    plt.show()

### Tensorboard for Colab

In [2]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc = TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://d5c26176.ngrok.io


### Logistic Regression

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
import time

#Defining the hyperparameters

learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

#Step 1: Read in data
mnist_folder = 'data/mnist'
download_mnist(mnist_folder)
train, val, test = read_mnist(mnist_folder, flatten = True)

#Step 2: Create datasets and iterator
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

#Create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

#Step 3: create weights and bias
w = tf.get_variable(name = 'weights', shape = [784, 10], initializer = tf.random_normal_initializer(mean = 0, stddev = 0.01))
b = tf.get_variable(name = 'bias', shape = [1, 10], initializer = tf.zeros_initializer())

#Step 4: build model
logits = tf.matmul(img, w) + b

#Step 5: define the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = label, logits = logits, name = 'entropy')
loss = tf.reduce_mean(entropy, name = 'loss')

#Step 6: define training operation
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

#Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds,1), tf.argmax(label,1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./Graph/logreg', tf.get_default_graph())
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    #train the model number of epochs times
    for i in range(n_epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Toal time : {0} seconds'.format(time.time() - start_time))
        
    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists
Average loss epoch 0: 0.36396381806495576
Average loss epoch 1: 0.29472465308946233
Average loss epoch 2: 0.283943564791319
Average loss epoch 3: 0.2781646569107854
Average loss epoch 4: 0.2736546872660171
Average loss epoch 5: 0.2742128196670566
Average loss epoch 6: 0.2704728376900041
Average loss epoch 7: 0.2655832806298899
Average loss epoch 8: 0.2627160980777685
Average loss epoch 9: 0.263606508318768
Average loss epoch 10: 0.26379387650725455
Average loss epoch 11: 0.2616857203633286
Average loss epoch 12: 0.26098569477366845
Average loss epoch 13: 0.2592407028688941
Average loss epoch 14: 0.2581491239022377
Average loss epoch 15: 0.25816859088318295
Average loss epoch 16: 0.25917671565399614
Average loss epoch 17: 0.2572768713325955
Average loss epoch 18: 0.256737866651180